In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
# Kaggle: https://www.kaggle.com/c/ieee-fraud-detection

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
test_identity = pd.read_csv("/kaggle/input/ieee-fraud-detection/test_identity.csv")
train_transaction = pd.read_csv("/kaggle/input/ieee-fraud-detection/train_transaction.csv")
train_identity = pd.read_csv("/kaggle/input/ieee-fraud-detection/train_identity.csv")
sample_submission = pd.read_csv("/kaggle/input/ieee-fraud-detection/sample_submission.csv")
test_transaction = pd.read_csv("/kaggle/input/ieee-fraud-detection/test_transaction.csv")
r_train=train_transaction.merge(train_identity, left_on="TransactionID",right_on="TransactionID", how="left",suffixes=('_tran','_iden'))
r_test=test_transaction.merge(test_identity, left_on="TransactionID",right_on="TransactionID", how="left",suffixes=('_tran','_iden'))
print ("{} dataset has {} rows(samples) with {} columns(features) each.".format('train',*r_train.shape))  
print ("{} dataset has {} rows(samples) with {} columns(features) each.".format('test',*r_test.shape))  

In [ ]:
#start H2O 
import h2o
from h2o.automl import H2OAutoML

#load data as h2o frames
h2o.init()
train = h2o.H2OFrame(r_train[0:1000])
test = h2o.H2OFrame(r_test[0:1000])

In [ ]:

#drop passengerId from data set
Id = test['TransactionID']
train = train.drop('TransactionID',axis =1)
test = test.drop('TransactionID',axis =1)
print('step1')
#identify predictors and labels
x = train.columns
y = 'isFraud'
x.remove(y)

In [ ]:


# Run AutoML
aml_ti = H2OAutoML(max_runtime_secs= 7200,max_models= 20, seed= 7,nfolds= 10)
aml_ti.train(x = x, y = y,
          training_frame = train)
          
#check the leaderboard
lb_ti = aml_ti.leaderboard
lb_ti


In [ ]:
pred = aml_ti.leader.predict(test)

In [ ]:
#save predict results to submission form
pred_df = pred.as_data_frame()
#pred_df['predict'].astype(int).head()
pred_res = pred_df['predict'].astype(int)
Id_df = Id.as_data_frame()
res_ti = pd.concat([Id_df,pred_res],axis=1,ignore_index = True)
res_ti.columns = ['TransactionID','isFraud']
print('step6')
res_ti.to_csv('subs.csv',index=False)
print('Complete')